In [1]:
import Pkg
Pkg.add("Pkg")
Pkg.add("Dates")

guypath = "/Users/guyfarmer/ambulance_connect/AmbulanceDeployement/AmbulanceDeployment.jl-legacy/"
michaelpath= "/home/mhilborn/Desktop/UT-Spring2021/SeniorLab/AmbulanceDeployement/AmbulanceDeployment.jl-legacy"

currentpath = michaelpath

cd(currentpath)
Pkg.activate(".")

using DataFrames, JLD, Distributions, CSV, Random, Plots,JSON
import DataStructures: PriorityQueue, enqueue!, dequeue!

include(".//src//model.jl")

   Updating registry at `~/.julia/registries/General`


   Updating git-repo `https://github.com/JuliaRegistries/General.git`


  Resolving package versions...
No Changes to `~/Desktop/UT-Spring2021/SeniorLab/AmbulanceDeployement/AmbulanceDeployment.jl-legacy/test/Project.toml`
No Changes to `~/Desktop/UT-Spring2021/SeniorLab/AmbulanceDeployement/AmbulanceDeployment.jl-legacy/test/Manifest.toml`
  Resolving package versions...
No Changes to `~/Desktop/UT-Spring2021/SeniorLab/AmbulanceDeployement/AmbulanceDeployment.jl-legacy/test/Project.toml`
No Changes to `~/Desktop/UT-Spring2021/SeniorLab/AmbulanceDeployement/AmbulanceDeployment.jl-legacy/test/Manifest.toml`
 Activating environment at `~/Desktop/UT-Spring2021/SeniorLab/AmbulanceDeployement/AmbulanceDeployment.jl-legacy/Project.toml`
  Resolving package versions...
No Changes to `~/Desktop/UT-Spring2021/SeniorLab/AmbulanceDeployement/AmbulanceDeployment.jl-legacy/Project.toml`
No Changes to `~/Desktop/UT-Spring2021/SeniorLab/AmbulanceDeployement/AmbulanceDeployment.jl-legacy/Manifest.toml`
  Resolving package versions...
No Changes to `~/Desktop/UT-Spring2021

reassign_ambulances! (generic function with 2 methods)

In [8]:
#=
Author : Ng Yeesian
Modified : Guy Farmer / Zander Tedjo / Will Worthington
generates the closest dispatch model
=#
mutable struct ClosestDispatch <: DispatchModel
    drivetime::DataFrame
    candidates::Vector{Vector{Int}}
    available::Vector{Int}
    assignment::Vector{Int} # which location the ambulance is assigned to
    ambulances::Vector{Vector{Int}} # list of ambulances assigned to each location
    status::Vector{Symbol} # the current status of the ambulance
        # possible statuses: :available, :responding, :atscene, :conveying, :returning
end

function ClosestDispatch(p::DeploymentProblem, drivetime::DataFrame,distribution::Vector{Int})
    candidates = Any[]
    I = 1:p.nlocations
    for region in 1:p.nregions
        push!(candidates, I[vec(p.coverage[region,:])])
    end
    println("creating the dispatch problem :$distribution")

    assignment = zeros(Int, p.nambulances)
    ambulances = [Int[] for i in 1:p.nlocations]
    k = 1
    for i in eachindex(distribution), j in 1:distribution[i]
        assignment[k] = i
        push!(ambulances[i], k)
        k += 1
    end
    status = fill(:available, p.nambulances)
    ClosestDispatch(drivetime, candidates, distribution, assignment, ambulances, status)

end

function ClosestDispatch(p::DeploymentProblem, problem::DispatchProblem)
    stn_names = [Symbol("stn$(i)_min") for i in 1:size(p.coverage,2)]
    ClosestDispatch(p, problem.emergency_calls[:, stn_names], problem.available)
end

function update_ambulances!(model::ClosestDispatch, i::Int, delta::Int)
    model.available[i] += delta
    @assert model.available[i] >= 0
end

function closest_available(dispatch::ClosestDispatch, id::Int, problem::DispatchProblem)
    location = 0
    min_time = Inf
    for i in dispatch.candidates[problem.emergency_calls[id, :neighborhood]]
        if problem.available[i] > 0 && dispatch.drivetime[id, i] < min_time
            println("apparently this station is available : $i and here is the amount of ambulances avalable: $(problem.available[i])")
            location = i
            min_time = dispatch.drivetime[id, i]
        end
    end
    location
end

function respond_to!(dispatch::ClosestDispatch, i::Int, t::Int)
    @assert length(dispatch.ambulances[i]) > 0 "$i: $(redeploy.ambulances[i])"
    amb = popfirst!(dispatch.ambulances[i])
    # @assert redeploy.hospital[amb] == 0
    @assert amb != 0
    @assert dispatch.status[amb] == :available "$amb: $(redeploy.status[amb])"
    dispatch.status[amb] = :responding
    #dispatch.fromtime[amb] = t
    amb
end

respond_to! (generic function with 2 methods)

In [14]:
turnaround = Distributions.LogNormal(3.65, 0.3)
lambda = 0
local_path = ""
model_name = "Stochastic"
namb = 40
ncalls = 1000
#using AmbulanceDeployment

pwd()

cd(currentpath * "/test/")
pwd()

#using AmbulanceDeployment
using Dates

#include("..//src//dispatch/closestdispatch.jl")
include("..//src//problem.jl")
include("..//src//evaluate.jl")
#include("..//src//simulate.jl")

hospitals = CSV.File(string(local_path,"../test/austin-data/hospitals.csv")) |> DataFrame
stations = CSV.File(string(local_path,"../test/austin-data/stations.csv")) |> DataFrame
hourly_calls = CSV.File("../test/austin-data/Full_WeekdayCalls.csv") |> DataFrame
adjacent_nbhd = CSV.File("../test/austin-data/adjacent_nbhd.csv") |> DataFrame
coverage = CSV.read("../test/austin-data/coverage_real.csv", DataFrame, header=false)
coverage = convert(Array{Bool, 2}, coverage[:, :])
incidents = CSV.File("../test/austin-data/austin_incidents.csv") |> DataFrame
solverstats = JLD.load("../src/outputs/austin_team_stats_1_03.jld")
amb_deployment = solverstats["amb_deployment"]
model_dict = Dict{String, Symbol}("Stochastic"=>:Stochastic, "Robust01"=>:Robust01, "Robust005"=>:Robust005, "Robust001"=>:Robust001, "Robust0001"=>:Robust0001,
          "Robust00001"=>:Robust00001, "MEXCLP"=>:MEXCLP, "MALP"=>:MALP)


p = DeploymentProblem(
          hourly_calls,
          adjacent_nbhd,
          coverage,
          namb = namb,
          train_filter = (hourly_calls[!,:year] .== 2019) .* (hourly_calls[!,:month] .<= 3)
          )
test_calls = CSV.File("../test/austin-data/austin_test_calls.csv")|> DataFrame
test_calls = test_calls[1:ncalls,:]
println("running $namb ambuances & $ncalls calls")

# distribution of ambulances across all of the stations
x = amb_deployment[model_dict[model_name]][namb]

#establishes the dispatch problem
problem = DispatchProblem(test_calls, hospitals, stations, p.coverage, x, turnaround=turnaround)

#establishes the dispatch model
dispatch = ClosestDispatch(p, problem)

Random.seed!(1234)



running 40 ambuances & 1000 calls
creating the dispatch problem :[1, 2, 1, 1, 1, 0, 1, 2, 2, 0, 1, 0, 0, 1, 1, 1, 4, 3, 1, 0, 0, 2, 0, 0, 1, 2, 0, 1, 1, 1, 1, 1, 1, 2, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1]


MersenneTwister(UInt32[0x000004d2], Random.DSFMT.DSFMT_state(Int32[-1393240018, 1073611148, 45497681, 1072875908, 436273599, 1073674613, -2043716458, 1073445557, -254908435, 1072827086  …  -599655111, 1073144102, 367655457, 1072985259, -1278750689, 1018350124, -597141475, 249849711, 382, 0]), [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], UInt128[0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000  …  0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x000000000000

In [15]:
using AmbulanceDeployment, Dates

In [16]:
struct EMSEngine{T}
    eventlog::DataFrame
    eventqueue::PriorityQueue{T,Int,Base.Order.ForwardOrdering}
    guiArray::Array{Any, 1}
end

struct gui_event
    #call made / call responded / ambulance arrived
    event_type::String
    #neighborhood where call is made from
    neighborhood_id::Int
    # station where call is made from
    deployment_id::Int
    # id of the event generated
    event_id::Int
    ambulance_id::Int
    arrival_time::Int
    remaining_amb::Int
    # current timestamp of event
    timestamp::DateTime
end

## generates the priority queue and instantiates the EMSEngine struct
function generateCalls(problem::DispatchProblem)
    ncalls = nrow(problem.emergency_calls)
    eventlog = DataFrame(
        id = 1:ncalls,
        dispatch_from = zeros(Int, ncalls),
        waittime = fill(0.0, ncalls),
        responsetime = fill(Inf, ncalls),
        scenetime = fill(Inf, ncalls),
        conveytime = fill(Inf, ncalls),
        returntime = fill(Inf, ncalls),
        return_to = zeros(Int, ncalls),
        return_type = fill(:station, ncalls),
        hospital = zeros(Int, ncalls),
        ambulance = zeros(Int, ncalls)
    )
    eventqueue = PriorityQueue{Tuple{Symbol,Int,Int,Int},Int}()
    guiArray = Any[]
    for i in 1:nrow(problem.emergency_calls)
        t = problem.emergency_calls[i, :arrival_seconds]
        enqueue!(eventqueue, (:call, i, t, problem.emergency_calls[i, :neighborhood]), t)
    end
    EMSEngine{Tuple{Symbol,Int,Int,Int}}(eventlog, eventqueue,guiArray) ## returns the ems engine struct
end

function simulate_events!(problem::DispatchProblem, dispatch::ClosestDispatch)
    engine = generateCalls(problem)
    while(~isempty(engine.eventqueue))
        (event, id, t, value) = dequeue!(engine.eventqueue)

        if event == :call
            # if the event is reachable then change event to station2call
            # if not it is a shortfall
            println("calling event id: $id time: $t value: $value")
            call_event!(engine, problem, dispatch, id, t, value)
           
        elseif event == :station2call
            #
        elseif event == :call2hospital

        elseif event == :hospital2station

        end
    end

end


function call_event!(
        ems::EMSEngine,
        problem::DispatchProblem,
        dispatch::DispatchModel,
        id::Int, # the id of the emergency call
        t::Int, # the time of the emergency call
        nbhd::Int; # the neighborhood the call is from
    )

    #check if there is an ambulance within the coverage matrix
    if sum(problem.deployment[problem.coverage[nbhd,:]]) == 0
        @assert false "$id: no ambulance reachable for call at $nbhd"
    #check if one of the ambulances within the coverage is available
    elseif sum(problem.available[problem.coverage[nbhd,:]]) > 0
        i = closest_available(dispatch, id, problem)
        println("closest available station : $i")
        @assert i > 0 "$id: dispatch from $i to nbhd $nbhd" # assume valid i (enforced by <if> condition)
        update_ambulances!(dispatch, i, -1)
        ems.eventlog[id, :dispatch_from] = i
        @assert problem.available[i] >= 0
        

        travel_time = ceil(Int, 60*2*problem.emergency_calls[id, Symbol("stn$(i)_min")])
        @assert travel_time >= 0
        ems.eventlog[id, :responsetime] = travel_time / 60 # minutes

        amb = respond_to!(dispatch, i, t)
        ems.eventlog[id, :ambulance] = amb

        event = gui_event("call responded", problem.emergency_calls[id, :neighborhood], i, id, amb,-1, problem.available[i] ,Dates.now())
        num_ambulances_array = Array{Integer}(undef, size(problem.available, 1))
        for i in 1:size(problem.available, 1)
            num_ambulances_array[i] = problem.available[i]
        end
        push!(ems.guiArray,event)
        push!(ems.guiArray,num_ambulances_array)
        enqueue!(ems.eventqueue, (:station2call, id, t + travel_time, amb), t + travel_time)
    #else queue it
    else
        # println(id, ": call from ", nbhd, " queued behind ", problem.wait_queue[nbhd])
        # event = gui_event("call made", nbhd, -1, id, -1,-1,-1,Dates.now())
        # push!(ems.guiArray,event)
        # problem.shortfalls = problem.shortfalls + 1
        # #push!(problem.shortfalls) # count shortfalls
        # push!(problem.wait_queue[nbhd], id) # queue the emergency call
        println(id, ": call from ", nbhd, " is a shortfall")
    end
end


call_event! (generic function with 1 method)

In [17]:
@time df, guiArray = simulate_events!(problem, dispatch)

calling event id: 1 time: 38 value: 83
apparently this station is available : 1 and here is the amount of ambulances avalable: 1
closest available station : 1
calling event id: 2 time: 575 value: 38
apparently this station is available : 19 and here is the amount of ambulances avalable: 1
apparently this station is available : 31 and here is the amount of ambulances avalable: 1
apparently this station is available : 43 and here is the amount of ambulances avalable: 1
closest available station : 43
calling event id: 3 time: 673 value: 129
apparently this station is available : 14 and here is the amount of ambulances avalable: 1
closest available station : 14
calling event id: 4 time: 691 value: 107
apparently this station is available : 34 and here is the amount of ambulances avalable: 2
closest available station : 34
calling event id: 5 time: 1166 value: 113
apparently this station is available : 4 and here is the amount of ambulances avalable: 1
apparently this station is available : 

77: call from 113 is a shortfall
calling event id: 78 time: 24714 value: 98
78: call from 98 is a shortfall
calling event id: 79 time: 25019 value: 98
79: call from 98 is a shortfall
calling event id: 80 time: 25961 value: 52
80: call from 52 is a shortfall
calling event id: 81 time: 26594 value: 114
81: call from 114 is a shortfall
calling event id: 82 time: 26642 value: 98
82: call from 98 is a shortfall
calling event id: 83 time: 26657 value: 115
83: call from 115 is a shortfall
calling event id: 84 time: 26802 value: 98
84: call from 98 is a shortfall
calling event id: 85 time: 27495 value: 50
85: call from 50 is a shortfall
calling event id: 86 time: 27777 value: 82
86: call from 82 is a shortfall
calling event id: 87 time: 27938 value: 50
87: call from 50 is a shortfall
calling event id: 88 time: 28192 value: 66
88: call from 66 is a shortfall
calling event id: 89 time: 29271 value: 128
89: call from 128 is a shortfall
calling event id: 90 time: 29501 value: 63
90: call from 63 i

201: call from 141 is a shortfall
calling event id: 202 time: 62498 value: 103
202: call from 103 is a shortfall
calling event id: 203 time: 62626 value: 83
203: call from 83 is a shortfall
calling event id: 204 time: 62710 value: 68
204: call from 68 is a shortfall
calling event id: 205 time: 63424 value: 51
205: call from 51 is a shortfall
calling event id: 206 time: 63450 value: 67
206: call from 67 is a shortfall
calling event id: 207 time: 63762 value: 66
207: call from 66 is a shortfall
calling event id: 208 time: 63937 value: 51
208: call from 51 is a shortfall
calling event id: 209 time: 64027 value: 35
209: call from 35 is a shortfall
calling event id: 210 time: 64403 value: 97
210: call from 97 is a shortfall
calling event id: 211 time: 64688 value: 67
211: call from 67 is a shortfall
calling event id: 212 time: 64875 value: 82
212: call from 82 is a shortfall
calling event id: 213 time: 66042 value: 108
213: call from 108 is a shortfall
calling event id: 214 time: 66215 valu

311: call from 98 is a shortfall
calling event id: 312 time: 86001 value: 98
312: call from 98 is a shortfall
calling event id: 313 time: 86125 value: 98
313: call from 98 is a shortfall
calling event id: 314 time: 86422 value: 83
314: call from 83 is a shortfall
calling event id: 315 time: 86453 value: 67
315: call from 67 is a shortfall
calling event id: 316 time: 86494 value: 124
316: call from 124 is a shortfall
calling event id: 317 time: 87157 value: 115
317: call from 115 is a shortfall
calling event id: 318 time: 87225 value: 52
318: call from 52 is a shortfall
calling event id: 319 time: 87236 value: 85
319: call from 85 is a shortfall
calling event id: 320 time: 87363 value: 56
320: call from 56 is a shortfall
calling event id: 321 time: 87741 value: 113
321: call from 113 is a shortfall
calling event id: 322 time: 87763 value: 67
322: call from 67 is a shortfall
calling event id: 323 time: 88484 value: 172
323: call from 172 is a shortfall
calling event id: 324 time: 88562 v

calling event id: 442 time: 134515 value: 129
442: call from 129 is a shortfall
calling event id: 443 time: 134539 value: 113
443: call from 113 is a shortfall
calling event id: 444 time: 134721 value: 113
444: call from 113 is a shortfall
calling event id: 445 time: 135132 value: 115
445: call from 115 is a shortfall
calling event id: 446 time: 135188 value: 113
446: call from 113 is a shortfall
calling event id: 447 time: 135404 value: 50
447: call from 50 is a shortfall
calling event id: 448 time: 135517 value: 65
448: call from 65 is a shortfall
calling event id: 449 time: 135672 value: 65
449: call from 65 is a shortfall
calling event id: 450 time: 135673 value: 65
450: call from 65 is a shortfall
calling event id: 451 time: 135755 value: 69
451: call from 69 is a shortfall
calling event id: 452 time: 135813 value: 82
452: call from 82 is a shortfall
calling event id: 453 time: 135843 value: 52
453: call from 52 is a shortfall
calling event id: 454 time: 135893 value: 82
454: call

calling event id: 572 time: 160623 value: 142
572: call from 142 is a shortfall
calling event id: 573 time: 161253 value: 83
573: call from 83 is a shortfall
calling event id: 574 time: 161516 value: 83
574: call from 83 is a shortfall
calling event id: 575 time: 162046 value: 113
575: call from 113 is a shortfall
calling event id: 576 time: 162176 value: 35
576: call from 35 is a shortfall
calling event id: 577 time: 162595 value: 113
577: call from 113 is a shortfall
calling event id: 578 time: 163122 value: 143
578: call from 143 is a shortfall
calling event id: 579 time: 163323 value: 98
579: call from 98 is a shortfall
calling event id: 580 time: 163681 value: 51
580: call from 51 is a shortfall
calling event id: 581 time: 163810 value: 128
581: call from 128 is a shortfall
calling event id: 582 time: 163928 value: 141
582: call from 141 is a shortfall
calling event id: 583 time: 164101 value: 141
583: call from 141 is a shortfall
calling event id: 584 time: 164896 value: 36
584: 

702: call from 98 is a shortfall
calling event id: 703 time: 202555 value: 113
703: call from 113 is a shortfall
calling event id: 704 time: 202605 value: 113
704: call from 113 is a shortfall
calling event id: 705 time: 202986 value: 83
705: call from 83 is a shortfall
calling event id: 706 time: 203940 value: 142
706: call from 142 is a shortfall
calling event id: 707 time: 205061 value: 82
707: call from 82 is a shortfall
calling event id: 708 time: 205157 value: 129
708: call from 129 is a shortfall
calling event id: 709 time: 205376 value: 113
709: call from 113 is a shortfall
calling event id: 710 time: 205510 value: 68
710: call from 68 is a shortfall
calling event id: 711 time: 206382 value: 68
711: call from 68 is a shortfall
calling event id: 712 time: 207486 value: 116
712: call from 116 is a shortfall
calling event id: 713 time: 207511 value: 67
713: call from 67 is a shortfall
calling event id: 714 time: 207780 value: 82
714: call from 82 is a shortfall
calling event id: 7

calling event id: 820 time: 237591 value: 67
820: call from 67 is a shortfall
calling event id: 821 time: 237761 value: 82
821: call from 82 is a shortfall
calling event id: 822 time: 237903 value: 82
822: call from 82 is a shortfall
calling event id: 823 time: 237948 value: 82
823: call from 82 is a shortfall
calling event id: 824 time: 237962 value: 98
824: call from 98 is a shortfall
calling event id: 825 time: 238116 value: 113
825: call from 113 is a shortfall
calling event id: 826 time: 238316 value: 97
826: call from 97 is a shortfall
calling event id: 827 time: 238476 value: 113
827: call from 113 is a shortfall
calling event id: 828 time: 238744 value: 157
828: call from 157 is a shortfall
calling event id: 829 time: 238851 value: 66
829: call from 66 is a shortfall
calling event id: 830 time: 238856 value: 66
830: call from 66 is a shortfall
calling event id: 831 time: 239079 value: 66
831: call from 66 is a shortfall
calling event id: 832 time: 239338 value: 98
832: call fro

calling event id: 951 time: 263745 value: 52
951: call from 52 is a shortfall
calling event id: 952 time: 263776 value: 128
952: call from 128 is a shortfall
calling event id: 953 time: 264027 value: 98
953: call from 98 is a shortfall
calling event id: 954 time: 264093 value: 141
954: call from 141 is a shortfall
calling event id: 955 time: 264735 value: 98
955: call from 98 is a shortfall
calling event id: 956 time: 264786 value: 128
956: call from 128 is a shortfall
calling event id: 957 time: 264821 value: 40
957: call from 40 is a shortfall
calling event id: 958 time: 265275 value: 51
958: call from 51 is a shortfall
calling event id: 959 time: 265917 value: 82
959: call from 82 is a shortfall
calling event id: 960 time: 266062 value: 68
960: call from 68 is a shortfall
calling event id: 961 time: 266455 value: 142
961: call from 142 is a shortfall
calling event id: 962 time: 266517 value: 68
962: call from 68 is a shortfall
calling event id: 963 time: 266708 value: 82
963: call f

LoadError: [91mMethodError: no method matching iterate(::Nothing)[39m
[91m[0mClosest candidates are:[39m
[91m[0m  iterate([91m::MathOptInterface.Bridges.Objective.Map[39m, [91m::Any...[39m) at /home/mhilborn/.julia/packages/MathOptInterface/ZJFKw/src/Bridges/Objective/map.jl:30[39m
[91m[0m  iterate([91m::MutableArithmetics.Zero[39m) at /home/mhilborn/.julia/packages/MutableArithmetics/bPWR4/src/rewrite.jl:88[39m
[91m[0m  iterate([91m::MutableArithmetics.Zero[39m, [91m::Nothing[39m) at /home/mhilborn/.julia/packages/MutableArithmetics/bPWR4/src/rewrite.jl:89[39m
[91m[0m  ...[39m